In [66]:
'''
This script reads the raw data from SiPM_simulator, filters each pulse with
digital filter and samples it. Filtered and sampled data are saved
'''

#!/usr/bin/env python
#-*- coding: utf-8 -*-
import sys
import os
import numpy as np
import pandas as pd
from scipy import signal
from random import randint

def main():
    acquisition_frequency = 1e+11 # Hz
    pad_int = 20000
    cut_off_frequency = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200, 250, 500]) * 1e+06 # Hz
    sample_frequency = np.array([100, 200, 500, 1000]) * 1e+06 # Hz

    path = "/home/sommema4/git/SiPM_simulator/data"
    path_out = "/home/sommema4/git/SiPM_simulator/data/preproc"

    for filename in os.listdir(path):
        file_in = os.path.join(path, filename)
        if filename.endswith(".csv"):
            sipm, scintillator, energy, particle = strip_filename(filename)
            dir_name = sipm + "_" + scintillator
            dir_out = os.path.join(path_out, dir_name)
            make_directory(dir_out)
            #print(file_in)
            df = read_csv(file_in)
            df_padded = pad_dataset(df, pad_int)
            for cut_off in cut_off_frequency:
                for sample in sample_frequency:
                    filename_out = str(cut_off * 1e-06) + "_" + str(sample * 1e-06) + ".csv"
                    file_out = os.path.join(dir_out, filename_out)
                    df_filter = filter_dataset(df_padded, cut_off, acquisition_frequency)
                    df_sample = sample_dataset(df_filter, sample, acquisition_frequency)
                    df_info = info_dataset(df_sample, energy, particle)
                    if os.path.isfile(file_out):
                        df.to_csv(file_out, mode='a', header=False) # append dataframe to file
                    else:
                        df.to_csv(file_out, header=False)
    return 0
    
def strip_filename(filename):
    base = os.path.basename(filename)
    name = os.path.splitext(base)[0]
    sipm, scintillator, energy, particle = name.split("_")
    return sipm, scintillator, energy, particle

def make_directory(dir_name):
    if not (os.path.isdir(dir_name)):
        try:
            os.makedirs(dir_name)
        except OSError:
            print ("Creation of the directory %s failed" % dir_name)
        else:
            print ("Successfully created the directory %s " % dir_name)

def read_csv(filename):
    # Read data to pandas dataframe and transpose it
    df = pd.read_csv(filename, sep=' ')
    df = df.iloc[:, :-1]
    return df

def pad_dataset(df, pad_int):
    pad = np.zeros((df.shape[0], pad_int))
    df_pad = pd.DataFrame(data=pad)
    df_out = pd.concat([df_pad, df], axis=1)
    return df_out

def noise_dataset(df):
    ss

def filter_wrap(x, b, a):
    x = x.fillna(0)
    return pd.Series(signal.filtfilt(b, a, x))

def filter_dataset(df, cut_off, freq_acq):
    # Construct and apply filter and amplification
    w = 2 * cut_off / freq_acq
    b, a = signal.butter(2, w, analog=False)
    df_out = df.apply(filter_wrap, axis=1, args=(b, a))
    return df_out

def sample_wrap(x, rep):
    start = randint(0, rep-1)
    return x[start::rep]

def sample_dataset(df, sample, freq_acq):
    rep = int(round(freq_acq / sample, 0)) 
    data = df.to_numpy()
    df_out = pd.DataFrame()
    for i in range(df.shape[0]//4):
        start = randint(0, rep-1)
        temp = pd.DataFrame(data[4*i:4*i+4, start::rep])
        if i == 0:
            df_out = df_out = temp
        else:
            df_out = df_out.append(temp, ignore_index = True)
    df_out[df_out < 1.0e-07] = 0
    return df_out

def info_dataset(df, energy, particle):
    temp = np.zeros((df.shape[0], 2))
    info = pd.DataFrame(data=temp, columns=["energy", "particle"])
    info.iloc[:,0] = energy
    info.iloc[:,1] = particle
    df_out = pd.concat([df, info], axis=1)
    print(df_out)
    return df_out
    
main()



              0             1             2             3             4  \
0  3.620845e-05  3.833338e-05  4.049756e-05  4.269570e-05  4.492189e-05   
1  1.193264e-05  1.316124e-05  1.444968e-05  1.579822e-05  1.720686e-05   
2  5.532257e-06  5.890770e-06  6.258284e-06  6.634121e-06  7.017502e-06   
3  3.609081e-07  4.167412e-07  4.760427e-07  5.388900e-07  6.053533e-07   

              5             6             7             8             9  ...  \
0  4.716956e-05  4.943140e-05  5.169942e-05  5.396482e-05  5.621801e-05  ...   
1  1.867539e-05  2.020331e-05  2.178982e-05  2.343383e-05  2.513389e-05  ...   
2  7.407547e-06  7.803263e-06  8.203547e-06  8.607169e-06  9.012774e-06  ...   
3  6.754943e-07  7.493653e-07  8.270079e-07  9.084523e-07  9.937156e-07  ...   

            262           263           264           265           266  \
0  7.491087e-07  7.462366e-07  7.440994e-07  7.426075e-07  7.416595e-07   
1  8.977328e-07  8.973478e-07  8.970329e-07  8.967934e-07  8.966288e-07  

KeyboardInterrupt: 